In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics

In [ ]:
df = pd.read_csv('mnist_train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB


In [ ]:
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB


In [ ]:
df = df.sample(frac=1, random_state=50)
y = df['label']
X = df.drop('label', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 50)

print(X_train.shape)
print(y_train.shape)

(48000, 784)
(48000,)


In [ ]:
# classifiers = {
#     # 'SVM': SVC(),
#     'KNN': KNeighborsClassifier(),
#     'Random Forest': RandomForestClassifier(random_state=50),
#     'Logistic Regression': LogisticRegression()
# }

# # Define hyperparameters grid for each classifier
# param_grids = {
#     # 'SVM': {
#     #     'C': [0.1, 1, 10, 100, 1000],
#     #     'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'],
#     #     'kernel': ['rbf','linear']
#     # },
#     'KNN': {
#         'n_neighbors': [3,5,7,9],
#         'weights': ['uniform', 'distance']
#     },
#     'Random Forest': {
#       'n_estimators': [50, 100, 150],
#       'max_depth': [None, 10, 20],
#       'min_samples_split': [2, 5, 10],
#       'max_features': ['sqrt', 'log2']
#     },
#     'HistGradientBoosting': {
#         'learning_rate': [0.1, 0.01, 0.001],
#         'max_iter': [50, 100, 200, 350, 500]
#     },
#     'Logistic Regression':{'solver': ['saga', 'liblinear'],
#                   'penalty': ['l1','l2'],
#                  'C': [0.001,0.01,0.1,1,10,100,1000],
#                   'max_iter' : [10,100,500]}
# }

In [ ]:
# results = {}
# for name, classifier in classifiers.items():
#     print("Training: ", name)
#     gridSearch = RandomizedSearchCV(estimator=classifier, param_distributions=param_grids[name], cv=5, n_jobs=-1)
#     gridSearch.fit(X_train, y_train)
#     results[name] = gridSearch.best_estimator_

# print(results)

In [ ]:
# for name, clf in results.items():
#     print("\nResults for", name)
#     y_pred = clf.predict(X_test)
#     accuracy = metrics.accuracy_score(y_test, y_pred)
#     print("Accuracy:", accuracy)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
model = Sequential()

# add input layer size to the first hidden layer
model.add(Dense(256, input_shape=(784,), activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation="relu"))
# output layer
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 256)               200960    
                                                                 
 dense_28 (Dense)            (None, 256)               65792     
                                                                 
 dense_29 (Dense)            (None, 256)               65792     
                                                                 
 dense_30 (Dense)            (None, 10)                2570      
                                                                 
Total params: 335114 (1.28 MB)
Trainable params: 335114 (1.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
learning_rate = 0.01

# stochastic gradient descent optimizer
sgd_optimizer = keras.optimizers.Adam()
# categoricall crossentropy loss function
cc_loss = keras.losses.CategoricalCrossentropy()

model.compile(optimizer=sgd_optimizer, loss=cc_loss, metrics=["accuracy"])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)


y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
encoder = OneHotEncoder(sparse_output=False).fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=64)
loss, accuracy = model.evaluate(X_test, y_test, batch_size=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
750/750 [==============================] - 8s 9ms/step - loss: 1.3100 - accuracy: 0.8791
Epoch 2/5
750/750 [==============================] - 7s 9ms/step - loss: 0.2234 - accuracy: 0.9449
Epoch 3/5
750/750 [==============================] - 6s 8ms/step - loss: 0.1617 - accuracy: 0.9565
Epoch 4/5
750/750 [==============================] - 7s 9ms/step - loss: 0.1253 - accuracy: 0.9650
Epoch 5/5
12000/12000 [==============================] - 23s 2ms/step - loss: 0.2035 - accuracy: 0.9567
Test Loss: 0.20353786647319794
Test Accuracy: 0.9566666483879089


In [ ]:
y_pred = model.predict(X_test)

375/375 [==============================] - 1s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = np.argmax(y_pred, axis = 1)
correct = np.argmax(y_test, axis = 1)
# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(correct, predictions))

# Compute additional metrics
print("\nClassification Report:")
print(classification_report(correct, predictions))

Confusion Matrix:
[[1208    0    3    0    1    2    2    2    1    1]
 [   0 1329    9    6    2    1    0    2    3    1]
 [  13    0 1175    6   12    1    1    6    2    3]
 [   4    3   16 1167    0   11    0   19   10    6]
 [   1    5    5    0 1093    2    3   13    2   20]
 [   4    0    4   23    2 1066   14    4   13   13]
 [  15    2    9    0    3    5 1116    0    6    2]
 [   1    5   10    3    1    0    0 1213    0    4]
 [   5    5   19   14   10   10    1    7 1071   18]
 [   6    3    1   13   30    4    0   28    3 1042]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1220
           1       0.98      0.98      0.98      1353
           2       0.94      0.96      0.95      1219
           3       0.95      0.94      0.95      1236
           4       0.95      0.96      0.95      1144
           5       0.97      0.93      0.95      1143
           6       0.98      0.96      0.97     